In [4]:
import numpy
import time
from tqdm import tqdm
import ipyparallel as ipp
from scipy import stats
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
#from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.circuit.library import RZZGate as rzz
from qiskit.circuit.library import RZGate as rz
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import AerSimulator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit.quantum_info import Pauli
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit.circuit import Parameter,ParameterVector
from qiskit import Aer
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, CircuitStateFn 
import scipy
from joblib import Parallel, delayed, Memory
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.circuit.random import random_circuit
from qiskit.chemistry.components.variational_forms import UCCSD

In [5]:
import warnings
warnings. filterwarnings('ignore')

In [6]:
c =ipp.Client()
num_qubits=8

In [7]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [8]:
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op

In [9]:
%run Utils.ipynb
%run optimizers.ipynb
%run EstimateHamiltonianMean.ipynb

In [10]:
U=7
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)

In [11]:
#Hamiltonian grouping for circuit based measurement
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate,circ0_1gate_inv,rotH2body=OffDiag2BodyTerms(opArr[2],send_inverse=True)
#one body off diags
circ0_8gate,circ0_8gate_inv,rotH1body=OffDiag1BodyTerms(opArr[0],send_inverse=True)
Ops=[rotH1body,opArr[1],rotH2body]

In [12]:
backend=Aer.get_backend('statevector_simulator')
HFcirc,HFstate=HF_init_state(U)

Final Hartree Fock Energy -18.788335371650774


In [13]:
a1=opArr[1][1].primitive.coeffs[0].real
a2=opArr[1][3].primitive.coeffs[0].real
a3=opArr[1][5].primitive.coeffs[0].real
a4=opArr[1][9].primitive.coeffs[0].real
a5=opArr[1][-3].primitive.coeffs[0].real
a6=opArr[2][0].primitive.coeffs[0].real
def H0circ(qc,p,i):
    qc.append(circ0_8gate,qr)
    qc.rz(p[i],qr[0])
    qc.rz(p[i],qr[1])
    qc.rz(p[i],qr[2])
    qc.rz(p[i],qr[3])
    qc.rz(-p[i],qr[4])
    qc.rz(-p[i],qr[5])
    qc.rz(-p[i],qr[6])
    qc.rz(-p[i],qr[7])
    qc.append(circ0_8gate_inv,qr)
    return qc
def H1circ(qc,p,i):
    qc.rz(a1*p[i],qr[2])
    qc.rz(a1*p[i],qr[3])
    qc.rz(a1*p[i],qr[6])
    qc.rz(a1*p[i],qr[7])
    qc.rz(a2*p[i],qr[0])
    qc.rz(a2*p[i],qr[1])
    qc.rz(a2*p[i],qr[4])
    qc.rz(a2*p[i],qr[5])
    qc.rzz(a3*p[i],qr[4],qr[5])
    qc.rzz(a3*p[i],qr[0],qr[1])
    qc.rzz(a4*p[i],qr[1],qr[5])
    qc.rzz(a4*p[i],qr[0],qr[4])
    qc.rzz(a4*p[i],qr[1],qr[5])
    qc.rzz(a4*p[i],qr[0],qr[4])
    qc.rzz(a5*p[i],qr[1],qr[4])
    qc.rzz(a5*p[i],qr[0],qr[5])
    return qc
def H2circ(qc,p,i):
    qc.append(circ0_1gate,[qr[0],qr[1]])
    qc.append(circ0_1gate,[qr[4],qr[5]])
    qc.rzz(a6*p[i],qr[1],qr[5])
    qc.rzz(a6*p[i],qr[0],qr[4])
    qc.rzz(-1*a6*p[i],qr[1],qr[4])
    qc.rzz(-1*a6*p[i],qr[0],qr[5])

    qc.append(circ0_1gate_inv,[qr[4],qr[5]])
    qc.append(circ0_1gate_inv,[qr[0],qr[1]])
    return qc

In [14]:
def AnsatzBuilder(p,Ops_labels):
    qc=QuantumCircuit(qr)
    for i in range(len(Ops_labels)):
        qc=pool[Ops_labels[i]](qc,p,i)  
    return qc   
def EnergyFromCirc(params):
    bound_qc=qc.bind_parameters({p: params})
    circ=QuantumCircuit(qr)
    circ.append(HFcirc,qr)
    circ.append(bound_qc.to_instruction(),qr)
    state_vector=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    E=(numpy.conjugate(state_vector)@Hmat@state_vector).real
    with open('SMOforHVA.txt','+a') as f:
        Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
        print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
    print(E)    
    return E
def fun_jac(params):
    setofParams1=[numpy.array(params)+numpy.array([0]*i+[0.001]+[0]*(len(params)-i-1)) for i in range(len(params))]
    setofParams2=[numpy.array(params)+numpy.array([0]*i+[-0.001]+[0]*(len(params)-i-1)) for i in range(len(params))]
    EnergySet1=list(map(EnergyFromCirc,setofParams1))
    EnergySet2=list(map(EnergyFromCirc,setofParams2))
    jac=(numpy.array(EnergySet1)-numpy.array(EnergySet2))/0.002
    return jac
pool=[H0circ,H1circ,H2circ]

In [15]:
def greedySearch(initial_params,num_points=100,eps=0.1):
    sets=[numpy.array(initial_params)+(2*numpy.random.rand(len(Ops_labels))-1)*eps for i in range(num_points)]
    Earr=[EnergyFromCirc(sets[i]) for i in tqdm(range(len(sets)))]
    Emin=min(Earr)
    params_int=numpy.array(sets)[numpy.array(Earr)==Emin][0]
    return params_int   

In [16]:
def SequentialOpt(cost,params,tol=1e-6):
    def costProfile(x):
        newParams=copy.deepcopy(params)
        newParams[ind:ind+1]=x
        return EnergyFromCirc(newParams)
    error=1000
    Eold=cost(params)
    print('start',Eold)
    intLoop=0
    while error>tol:
        print("looped",intLoop)
        intLoop+=1
        for i in range(len(params)):
            #coeffs=determine_unknowns(cost,params,i)
            ind=i
            res = scipy.optimize.minimize(costProfile, params[ind], method='L-BFGS-B',tol=1e-6)
            params[ind]=res['x'][0]
            E=res['fun']
            print(E)
            with open('SMOforHVA.txt','+a') as f:
                Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
        error=numpy.abs(Eold-E)
        print(error)
        Eold=E
    return params,E

In [18]:
#Two layer ansatz
Ops_labels=[0,1,2]*2
qr=QuantumRegister(8,'qr')
p=ParameterVector('p',len(Ops_labels))
qc=AnsatzBuilder(p,Ops_labels)

In [19]:
params_int=greedySearch([0.0]*len(Ops_labels),200)

  5%|▌         | 10/200 [00:04<01:26,  2.21it/s]

KeyboardInterrupt: 

In [41]:
EnergyFromCirc(params_int)

-18.881438206530753

In [42]:
SequentialOpt(EnergyFromCirc,params_int)

start -18.881438206530753
looped 0
-18.882235057566447
-18.901521343196144
-18.903035580627947
-18.965808614830106
-18.98738276437387
-18.9914231869393
0.10998498040854798
looped 1
-18.992022102075317
-18.99339199085049
-18.993642225716208
-18.99425802869986
-18.994465927954785
-18.994566928752068
0.003143741812767331
looped 2
-18.99553306412987
-18.996050713123296
-18.9961451893022
-18.996502227372194
-18.9973095426323
-18.9973983023744
0.0028313736223317676
looped 3
-18.998233571142258
-18.999080034858405
-18.999151034677585
-18.999504443340093
-19.000464270092287
-19.000548030179385
0.003149727804984792
looped 4
-19.001491150936968
-19.002568283455503
-19.002648192914414
-19.002981791426365
-19.004149184317807
-19.004238205400544
0.003690175221159109
looped 5
-19.005268276044237
-19.006596034174724
-19.006685790338388
-19.00697464297602
-19.008353893795427
-19.008441010138505
0.0042028047379609745
looped 6
-19.00952965455996
-19.011091390052147
-19.01118155252782
-19.01140283077224


(array([ 0.07646747,  0.35193527,  0.25707313,  0.14685232, -0.53501084,
        -0.27573109]),
 -19.03530525799791)

In [44]:
#three layer ansatz
Ops_labels=[0,1,2,0,1,2,0,1,2]
qr=QuantumRegister(8,'qr')
p=ParameterVector('p',len(Ops_labels))
qc=AnsatzBuilder(p,Ops_labels)

In [45]:
#choosing parameters corresponding to lowest energy value in set
with open('SMOforHVA.txt','r') as f:
    lines=f.readlines()
    paramsFromLast=eval(lines[-1].split('#')[0])
    
params_int=greedySearch(paramsFromLast+[0]*3,250,0.1)

100%|██████████| 250/250 [04:12<00:00,  1.01s/it]


In [50]:
SequentialOpt(EnergyFromCirc,params_int)

start -19.044363602922974
looped 0
-19.045497493629323
-19.046786519084524
-19.04742917514742
-19.04861879320242
-19.052054522144342
-19.056061359636843
-19.05804342194204
-19.05920216027482
-19.062322702307835
0.017959099384860622
looped 1
-19.062366742269372
-19.067754425352515
-19.0680091716853
-19.068100889074916
-19.071525588718565
-19.075343826188423
-19.076002508224338
-19.07606871467045
-19.077372447960187
0.015049745652351731
looped 2
-19.077489862092715
-19.081146444089935
-19.082766217085965
-19.082823197995488
-19.087545499903634
-19.090493258093854
-19.091044445265656
-19.091305089072232
-19.091831867232784
0.014459419272597529
looped 3
-19.091835883873404
-19.095058089114882
-19.09688541789771
-19.09715784681274
-19.102264324314095
-19.104338464634342
-19.104803379610836
-19.105183484505606
-19.105534156822763
0.013702289589978989
looped 4
-19.105534890194694
-19.108122744120788
-19.109537216211486
-19.11002596334919
-19.114476133358224
-19.115795994154883
-19.11615432193

KeyboardInterrupt: 

In [24]:
with open('SMOforHVA.txt','r') as f:
    lines=f.readlines()
    paramsFromLast=eval(lines[-1].split('#')[0])

In [44]:
#Four layer ansatz
Ops_labels=[0,1,2,0,1,2,0,1,2,0,1,2]
qr=QuantumRegister(8,'qr')
p=ParameterVector('p',len(Ops_labels))
qc=AnsatzBuilder(p,Ops_labels)

In [54]:
params_int=greedySearch(paramsFromLast,300,0.01)#+[0]*3,100,0.008)

100%|██████████| 300/300 [02:40<00:00,  1.87it/s]


In [56]:
EnergyFromCirc(params_int)

-19.177459890252262

In [57]:
SequentialOpt(EnergyFromCirc,params_int)

start -19.177459890252262
looped 0
-19.17746448694813
-19.1774653637067
-19.177538503612194
-19.177539086838244
-19.177561712082014
-19.177565071614982
-19.177644874824924
-19.17768194909135
-19.1776831954084
-19.17768544456757
-19.177708898030982
-19.177711385415282
0.00025149516302036545
looped 1
-19.177716732685003
-19.177743435977074
-19.177745330014158
-19.177746717361405
-19.17776498286228
-19.17776884936205
-19.177772295155393
-19.177781990026578
-19.177785183933977
-19.177785593470247
-19.177814577076305
-19.17781484616222
0.00010346074693856622
looped 2
-19.177815625017892
-19.17784276434842
-19.17784682664256
-19.177849961697735
-19.17786741441136
-19.177871068935943
-19.177871109942764
-19.17787211180581
-19.177874378314677
-19.17787439659216
-19.177883697083253
-19.1778851349818
7.028881957893418e-05
looped 3
-19.177890022699412
-19.177916859865437
-19.17791922469844
-19.177921560991017
-19.17793137397917
-19.17793402593144
-19.17793418554539
-19.177935282675648
-19.1779360

-19.179292532174006
-19.179292627064907
-19.17929273263822
-19.17930299366257
-19.17930475801898
3.735726057385591e-05
looped 31
-19.179313386259203
-19.1793213457088
-19.17932419827475
-19.179328000020597
-19.1793286245143
-19.179328766636285
-19.179329083976125
-19.179329289037778
-19.179329371883725
-19.179329462854152
-19.17933924963865
-19.179341029841584
3.627182260501627e-05
looped 32
-19.179349479616494
-19.179357058994547
-19.179359891603443
-19.179363712959265
-19.179364327301887
-19.17936447269597
-19.179364782923336
-19.179364975053893
-19.179365049458298
-19.179365125282274
-19.1793744566341
-19.179376250132982
3.52202913980193e-05
looped 33
-19.179384527269974
-19.179391751377327
-19.179394552359124
-19.179398379893772
-19.179398979248127
-19.179399132536147
-19.17939942891528
-19.179399606181597
-19.179399670987916
-19.1793997354258
-19.17940865145243
-19.17941044483123
3.4194698248057875e-05
looped 34
-19.17941855836203
-19.17942544621471
-19.179428216982775
-19.1794320

-19.18007585853573
-19.180079051100865
-19.18007908552876
-19.180079308878316
-19.180079390181568
-19.180079406986604
-19.180079428575713
-19.180079436739316
-19.180082533291486
-19.18008368763065
1.6915254921201495e-05
looped 62
-19.180088747265856
-19.180090960596058
-19.180092556102977
-19.18009572042691
-19.180095755479663
-19.18009597386817
-19.180096051113537
-19.180096066603582
-19.18009608756085
-19.180096096620723
-19.18009909663877
-19.18010021404472
1.6526414071904583e-05
looped 63
-19.180105176091793
-19.180107312678373
-19.180108868827073
-19.180112009377492
-19.180112044293736
-19.180112260725178
-19.180112334349115
-19.18011234832088
-19.18011236829
-19.180112378182823
-19.180115286470283
-19.18011637781267
1.6163767948995655e-05
looped 64
-19.180121246079576
-19.180123308229774
-19.180124826186184
-19.180127927780983
-19.180127962147367
-19.180128174794312
-19.180128244625113
-19.18012825738971
-19.18012827695235
-19.18012828771985
-19.180131107221378
-19.18013216930029

-19.180455695985554
9.237870695955053e-06
looped 92
-19.180458694156837
-19.180459534992263
-19.180460326929904
-19.180462697668126
-19.18046272292232
-19.18046283770155
-19.18046285838968
-19.18046286060529
-19.180462871001264
-19.180462890534518
-19.180464234255222
-19.180464774041006
9.0780554522496e-06
looped 93
-19.180467726749843
-19.180468545269818
-19.1804693174155
-19.18047166219668
-19.180471686376382
-19.180471800452832
-19.18047182059319
-19.18047182266347
-19.180471833066264
-19.18047185309101
-19.180473162325523
-19.18047368844583
8.914404823201494e-06
looped 94
-19.180476594489438
-19.180477388263643
-19.18047814148706
-19.180480461791536
-19.180480485517055
-19.180480596575784
-19.1804806153261
-19.180480617123223
-19.180480627139243
-19.180480646733564
-19.180481930122188
-19.18048244921253
8.76076670053294e-06
looped 95
-19.180485310926365
-19.180486083184135
-19.180486825126668
-19.180489126703105
-19.180489150180946
-19.180489258701705
-19.180489277117413
-19.180489

KeyboardInterrupt: 

In [36]:
#Five layer ansatz
Ops_labels=[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]
qr=QuantumRegister(8,'qr')
p=ParameterVector('p',len(Ops_labels))
qc=AnsatzBuilder(p,Ops_labels)

In [30]:
params_int=greedySearch(paramsFromLast+[0]*3,500,0.007)#+[0]*3,100,0.008)

100%|██████████| 500/500 [05:01<00:00,  1.66it/s]


In [18]:
EnergyFromCirc(params_int)

-19.183299716074213

In [32]:
SequentialOpt(EnergyFromCirc,params_int)

start -19.180663067345506
looped 0
-19.180670185364686
-19.180688216547914
-19.180689570745987
-19.18069539717494
-19.180696241868077
-19.18070784674515
-19.180707849715535
-19.18072876623444
-19.18073399866701
-19.180734451938687
-19.180735840013718
-19.18074934154387
-19.18082888472061
-19.180850921000506
-19.180852782785234
0.0001897154397276779
looped 1
-19.180899711191955
-19.180900665843122
-19.180904884440608
-19.180906117161168
-19.18090694916604
-19.180917105823422
-19.180917123656222
-19.180922273369166
-19.180925670091458
-19.180926461001697
-19.180930412664498
-19.18093184959091
-19.18100019652065
-19.181017531429223
-19.181018631648026
0.00016584886279247257
looped 2
-19.181068359883646
-19.181069581718372
-19.18107239827291
-19.181073194557708
-19.181073199324185
-19.181079819776098
-19.181079840418096
-19.18107999691433
-19.1810802010845
-19.18108147328173
-19.181087645383478
-19.181088447325667
-19.181150872090964
-19.181165410423585
-19.1811664572592
0.0001478256111724

-19.18263620089206
-19.182636805681327
-19.18263746919593
-19.18263749811688
-19.182639342792395
-19.18263940819314
-19.182639410648026
-19.182639433580388
-19.182641101942888
-19.182641393452467
-19.182641804342932
-19.18265016125619
-19.182650795245785
-19.182650828967503
2.733168019020127e-05
looped 26
-19.182662808799115
-19.182662975707288
-19.18266354721493
-19.18266419184476
-19.182664216243293
-19.1826660297091
-19.182666089296553
-19.182666090727423
-19.182666106249233
-19.18266769014782
-19.18266797316507
-19.18266838777361
-19.18267635596779
-19.18267695982629
-19.182676986127102
2.6157159599193847e-05
looped 27
-19.1826884498987
-19.182688603512112
-19.182689144137548
-19.182689775343395
-19.182689796442723
-19.182691571262325
-19.182691624597
-19.182691625918828
-19.182691635998978
-19.182693142246933
-19.182693418775113
-19.182693837718666
-19.182701441421347
-19.182702016186955
-19.182702036699947
2.5050572844520502e-05
looped 28
-19.182713004502745
-19.182713146123326
-

-19.183063851659774
-19.183063861059015
-19.183064844434067
-19.183064874785483
-19.18306487505715
-19.18306487508235
-19.183065581257882
-19.18306570083236
-19.183066383277506
-19.183068887718463
-19.18306907031206
-19.18306910161654
1.0116557525918779e-05
looped 51
-19.18307333862819
-19.183073366616117
-19.183073421942563
-19.183073784710036
-19.183073794051612
-19.183074745001218
-19.183074774668547
-19.183074774924012
-19.183074774967565
-19.18307545973927
-19.18307557349089
-19.18307626573063
-19.183078648306797
-19.183078824876393
-19.183078859642013
9.758025473871612e-06
looped 52
-19.183082931610667
-19.183082956937778
-19.183083002623256
-19.183083359993677
-19.183083369026267
-19.18308430184966
-19.183084331165855
-19.18308433134394
-19.1830843313725
-19.183084995180984
-19.18308510247573
-19.1830858041426
-19.183088085057197
-19.183088253713
-19.183088292082193
9.432440180034973e-06
looped 53
-19.183092217315348
-19.18309224170398
-19.1830922801189
-19.183092630548742
-19.1

-19.183239187040027
-19.1832391872435
-19.183239187870495
-19.183239561158764
-19.183239592284924
-19.183240459259085
-19.18324129880449
-19.183241367604566
-19.183241489311843
4.899049518058973e-06
looped 76
-19.183243234215
-19.18324324313237
-19.18324325957842
-19.183243501681332
-19.183243509069055
-19.18324399140205
-19.183244014977607
-19.183244015143448
-19.183244015793516
-19.183244378922574
-19.183244409307502
-19.183245280978078
-19.18324608883314
-19.183246154168753
-19.183246280053364
4.790741520821484e-06
looped 77
-19.183247972608328
-19.18324798081177
-19.183247999249794
-19.1832482404777
-19.183248247846294
-19.18324871519006
-19.183248739115307
-19.183248739302407
-19.18324874017112
-19.183249096454144
-19.183249125287187
-19.183250009861368
-19.18325078801352
-19.183250850246484
-19.1832509784712
4.698417836124236e-06
looped 78
-19.18325261580039
-19.183252623351464
-19.183252644816566
-19.18325288470684
-19.183252892113035
-19.18325334774369
-19.18325337049715
-19.18

-19.183336802152446
-19.183336821721788
-19.183337770387794
-19.18333812625032
-19.1833381419077
-19.183338333648397
3.2071547551026924e-06
looped 101
-19.18333915327043
-19.18333915561049
-19.183339257993133
-19.18333946479291
-19.183339470872497
-19.18333973452244
-19.183339753400226
-19.183339754035746
-19.183339755083445
-19.183339980741888
-19.183340000536486
-19.183340948467908
-19.183341290447927
-19.18334130493435
-19.183341491340553
3.1576921557530113e-06
looped 102
-19.18334228945644
-19.183342291769936
-19.1833423967625
-19.183342603864624
-19.183342609313378
-19.183342866638874
-19.18334288530893
-19.18334288593168
-19.183342886999124
-19.183343109541887
-19.183343128788398
-19.18334408038532
-19.183344418034117
-19.18334443125452
-19.183344622252363
3.130911810700354e-06
looped 103
-19.18334539595018
-19.18334539807898
-19.183345506411246
-19.183345712471702
-19.18334571833136
-19.183345967734912
-19.183345986208714
-19.183345986868602
-19.183345987897535
-19.1833462053421

-19.183407386650618
-19.183408350385772
-19.18340854327468
-19.1834085434872
-19.18340876614743
2.5508037140298256e-06
looped 126
-19.18340917943001
-19.183409179660735
-19.18340935741412
-19.18340956157392
-19.18340956697628
-19.183409717117062
-19.183409732210873
-19.18340973354458
-19.183409734303773
-19.18340989616929
-19.183409920072403
-19.183410889570606
-19.18341107908242
-19.183411079210934
-19.183411305052235
2.5389048055046715e-06
looped 127
-19.18341170827981
-19.183411708518882
-19.183411890270655
-19.183412096569707
-19.18341210185064
-19.183412249985928
-19.1834122643811
-19.18341226554302
-19.183412266246204
-19.18341242557465
-19.183412449951124
-19.183413415537153
-19.18341360307421
-19.183413603112328
-19.183413826579415
2.521527179055738e-06
looped 128
-19.1834142197649
-19.183414219999452
-19.183414401190838
-19.183414607955974
-19.183414613069814
-19.18341475750836
-19.183414772221802
-19.183414773385348
-19.18341477417499
-19.183414932057378
-19.183414956708244
-

KeyboardInterrupt: 

In [38]:
#Six layer ansatz
Ops_labels=[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]
qr=QuantumRegister(8,'qr')
p=ParameterVector('p',len(Ops_labels))
qc=AnsatzBuilder(p,Ops_labels)

In [35]:
with open('SMOforHVA.txt','r') as f:
    lines=f.readlines()
    paramsFromLast=eval(lines[-1].split('#')[0])

In [37]:
EnergyFromCirc(paramsFromLast)

-19.183459720859908

In [23]:
params_int=greedySearch(paramsFromLast+[0]*3,700,0.003)

100%|██████████| 700/700 [09:50<00:00,  1.18it/s]


In [42]:
AdaDelta_sv(EnergyFromCirc,paramsFromLast+[0.0]*3,eps0=5e-11,runs=50,tol=1e-5,save_opt_steps=True)

  0%|          | 0/18 [00:00<?, ?it/s]

0 Index


  0%|          | 0/18 [00:00<?, ?it/s]

updated energy -19.183459992973464 learning rate [0.01225841 0.06555142 0.49439983 0.00633326 0.01507612 0.05482749
 0.03282502 0.01512105 0.09232944 0.00607063 0.73670959 0.04145827
 0.57447869 0.08090147 0.61917892 0.06992421 0.08090147 0.6191789 ]
conv err 0.9000000272113574
1 Index


  0%|          | 0/18 [00:00<?, ?it/s]

...

updated energy -19.183461697764937 learning rate [0.02       0.02001797 0.02001797 0.02001797 0.02001797 0.02
 0.02001797 0.02001797 0.02001797 0.02001797 0.02001797 0.02001797
 0.02001797 0.02       0.02       0.02001797 0.02       0.02      ]
conv err 0.025031820977557954
35 Index


  0%|          | 0/18 [00:00<?, ?it/s]

updated energy -19.183461935352998 learning rate [0.02000929 0.02000929 0.02       0.02000929 0.02000929 0.02000929
 0.02000929 0.02000929 0.02000929 0.02000929 0.02000929 0.02
 0.02000929 0.02000929 0.02000929 0.02000929 0.02000929 0.02000929]
conv err 0.022528662638608282
36 Index


 33%|███▎      | 6/18 [00:28<00:57,  4.76s/it]


KeyboardInterrupt: 

In [17]:
#Seven layer ansatz
Ops_labels=[0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]
qr=QuantumRegister(8,'qr')
p=ParameterVector('p',len(Ops_labels))
qc=AnsatzBuilder(p,Ops_labels)

In [20]:
with open('../7/FinalDataFiles/HVA-opt-SMO&AdaDelta/SMOforHVA.txt','r') as f:
    lines=f.readlines()
    paramsFromLast=eval(lines[-1].split('#')[0])

In [61]:
SequentialOpt(EnergyFromCirc,paramsFromLast)

start -19.183543925051058
looped 0
-19.18354406790604
-19.183544119787637
-19.183544131990015


KeyboardInterrupt: 

In [33]:
res=scipy.optimize.minimize(EnergyFromCirc,paramsFromLast)

-19.183541503419498
-19.183541503419498
-19.183541503400455
-19.18354150342103
-19.18354150341063
-19.183541503348184
-19.18354150338368
-19.1835415034105
-19.18354150338203
-19.18354150339075
-19.183541503411522
-19.183541503361777
-19.183541503416812
-19.183541503424024
-19.18354150341267
-19.183541503419402
-19.183541503419434
-19.183541503418546
-19.183541503419487
-19.18354150341945
-19.183541503418247
-19.1835415034196
-19.18354150341947
-19.181795420317616
-19.181795420317616
-19.18179542004856
-19.181795423047717
-19.181795420562853
-19.181795424300866
-19.181795422918924
-19.18179542057427
-19.181795423460642
-19.18179542292348
-19.181795420599357
-19.18179542380418
-19.18179542161869
-19.181795420439308
-19.181795422166626
-19.18179542010371
-19.18179542029744
-19.181795423182724
-19.181795420112177
-19.1817954202984
-19.181795423167106
-19.18179542011773
-19.18179542029908
-19.183541952417173
-19.183541952417173
-19.18354195239379
-19.183541952461706
-19.183541952412256
-19.

-19.183807491346798
-19.1838074914556
-19.1838074915581
-19.183807491002817
-19.183807491527165
-19.18380749156736
-19.183807491098314
-19.183807491502165
-19.183807491551903
-19.183807491342442
-19.18380749162133
-19.18380749155165
-19.18380749137657
-19.18380749162501
-19.183807491551104
-19.183807491368633
-19.183807491630297
-19.18380749154998
-19.18385616072341
-19.18385616072341
-19.183856161001973
-19.18385616013981
-19.1838561606606
-19.183856159910057
-19.183856160135182
-19.183856160676722
-19.183856159689
-19.183856160133757
-19.183856160668444
-19.183856159555145
-19.183856160192832
-19.183856160672
-19.183856160132475
-19.183856160890446
-19.18385616073633
-19.183856160138617
-19.183856160879316
-19.183856160734283
-19.183856160301335
-19.183856160762453
-19.183856160720218
-19.183877994573855
-19.183877994573855
-19.183877994698367
-19.18387799431005
-19.183877994540282
-19.183877994143376
-19.18387799429554
-19.183877994556507
-19.183877993895976
-19.183877994342478
-19.

-19.184309223555516
-19.184309223450352
-19.184309223718152
-19.18430922377563
-19.184309223463366
-19.184309223156298
-19.184309223532424
-19.18430922348654
-19.18430922325927
-19.184309223317868
-19.184309223481275
-19.184309223185128
-19.18430922349696
-19.184309223466027
-19.184349153266
-19.184349153266
-19.18434915311694
-19.18434915340197
-19.184349153250658
-19.18434915340515
-19.184349153355097
-19.1843491532468
-19.184349153352343
-19.184349153405098
-19.18434915329831
-19.184349153570867
-19.18434915357864
-19.184349153289638
-19.184349153096075
-19.184349153341138
-19.184349153288398
-19.184349153166995
-19.184349153135717
-19.184349153267593
-19.184349153081097
-19.184349153326433
-19.18434915327646
-19.18440891517744
-19.18440891517744
-19.184408915131346
-19.184408915258967
-19.18440891516918
-19.184408915243218
-19.184408915254117
-19.18440891516846
-19.184408915278134
-19.184408915262186
-19.18440891521433
-19.184408915324664
-19.184408915298206
-19.18440891518486
-19.

-19.184547923347143
-19.184547923270014
-19.184547923394383
-19.18454792334986
-19.184547923256876
-19.18454792339471
-19.18454792334146
-19.184547923252172
-19.184547923372676
-19.184547923343953
-19.184552459387184
-19.184552459387184
-19.184552459426992
-19.18455245936057
-19.18455245937916
-19.18455245937012
-19.18455245934082
-19.184552459376967
-19.184552459279622
-19.184552459343518
-19.184552459380342
-19.184552459438216
-19.18455245944749
-19.184552459402667
-19.184552459269693
-19.18455245947681
-19.184552459406536
-19.184552459266026
-19.18455245944983
-19.18455245939351
-19.184552459244834
-19.184552459442763
-19.184552459397583
-19.184558376726795
-19.184558376726795
-19.184558376778966
-19.18455837671355
-19.184558376716144
-19.184558376738995
-19.184558376690635
-19.18455837671238
-19.184558376607423
-19.184558376697396
-19.184558376715287
-19.184558376812035
-19.184558376823635
-19.18455837674922
-19.18455837658834
-19.184558376842343
-19.184558376754083
-19.18455837659

-19.184685573892992
-19.184685574328956
-19.184685574161453
-19.18468557392422
-19.184685574191263
-19.184685574126977
-19.184698197525346
-19.184698197525346
-19.184698197589974
-19.184698197497774
-19.18469819751827
-19.184698197443286
-19.18469819747514
-19.184698197518603
-19.18469819726868
-19.184698197440866
-19.184698197505273
-19.184698197457802
-19.184698197429075
-19.184698197513853
-19.184698197531567
-19.184698197545167
-19.1846981975163
-19.184698197491883
-19.184698197688412
-19.184698197531223
-19.1846981975607
-19.184698197560586
-19.184698197519236
-19.18471571539873
-19.18471571539873
-19.18471571543204
-19.1847157152955
-19.18471571538242
-19.184715715341675
-19.184715715266226
-19.184715715380875
-19.184715715277502
-19.18471571525801
-19.18471571536775
-19.184715715376168
-19.18471571534209
-19.18471571538649
-19.184715715308368
-19.184715715519648
-19.18471571539799
-19.184715715253752
-19.184715715635946
-19.184715715417045
-19.18471571531437
-19.184715715436404


...

-19.184963235658497
-19.18496323568526
-19.184963235664696
-19.184963235688596
-19.184963235684137
-19.18496323566412
-19.18496323566754
-19.184963235691693
-19.184963235663346
-19.184963235666604
-19.18496323566594
-19.184963235662934
-19.184963235661648
-19.18496323564649
-19.184963235664505
-19.184963235676932
-19.184963235632633
-19.184963235669915
-19.184963235690617
-19.18496323566221
-19.184963235665673
-19.184963785761358
-19.184963785761358
-19.184963785771895
-19.18496378574205
-19.18496378575671
-19.184963785772926
-19.18496378573456
-19.184963785755397
-19.18496378573533
-19.18496378573761
-19.184963785754643
-19.18496378575837
-19.184963785746554
-19.184963785757255
-19.184963785799415
-19.184963785755635
-19.18496378575939
-19.184963785782667
-19.18496378581225
-19.18496378576774
-19.18496378579596
-19.184963785764634
-19.18496378575958
-19.18496398937203
-19.18496398937203
-19.184963989378435
-19.184963989379263
-19.184963989370207
-19.184963989374342
-19.184963989375973

-19.184979760205458
-19.184979760195098
-19.18497976018414
-19.184979760208662
-19.18497976019616
-19.184979760223918
-19.184979760198814
-19.18497976019565
-19.184979760205813
-19.184979760203152
-19.18497976019369
-19.184979760214425
-19.184979760184497
-19.184979760184458
-19.184979760203184
-19.18497976019383
-19.18497976019124
-19.184980280053793
-19.184980280053793
-19.18498028006785
-19.184980280042616
-19.184980280054965
-19.18498028007631
-19.184980280037937
-19.184980280055402
-19.184980280021882
-19.18498028002962
-19.184980280058124
-19.18498028008497
-19.18498028003842
-19.184980280054827
-19.184980280096546
-19.184980280053075
-19.18498028005228
-19.18498028009134
-19.184980280084332
-19.184980280048055
-19.184980280089768
-19.184980280060177
-19.184980280052308
-19.184980698933508
-19.184980698933508
-19.184980698943455
-19.184980698920835
-19.18498069893311
-19.18498069894059
-19.184980698916245
-19.184980698933185
-19.18498069890991
-19.184980698907154
-19.184980698934

-19.18498499841715
-19.184984998422234
-19.184984998428234
-19.184984998419093
-19.18498499842179
-19.184984998426536
-19.18498499843011
-19.184984998421992
-19.184984998429197
-19.18498499842182
-19.184984998422244
-19.184985008347997
-19.184985008347997
-19.18498500834809
-19.184985008347578
-19.184985008347905
-19.184985008347006
-19.184985008347372
-19.18498500834783
-19.184985008346096
-19.184985008347276
-19.18498500834774
-19.18498500834513
-19.184985008346025
-19.18498500834776
-19.184985008350008
-19.18498500834596
-19.18498500834769
-19.184985008349624
-19.184985008349248
-19.184985008348104
-19.1849850083508
-19.184985008347347
-19.184985008348086
-19.184985014159828
-19.184985014159828
-19.184985014160034
-19.184985014159572
-19.18498501415977
-19.1849850141583
-19.18498501415937
-19.184985014159786
-19.184985014159057
-19.18498501415925
-19.184985014159665
-19.184985014157068
-19.18498501415833
-19.184985014159647
-19.184985014161363
-19.18498501415827
-19.184985014159594


-19.184985037251355
-19.18498503725136
-19.184985037251383
-19.18498503725157
-19.184985037251316
-19.18498503725139
-19.184985037251312
-19.18498503725134
-19.184985037251497
-19.184985037251497
-19.18498503725147
-19.18498503725129
-19.184985037251515
-19.184985037251504
-19.184985037251312
-19.18498503725151
-19.184985037251295
-19.184985037251337
-19.184985037251536
-19.18498503725129
-19.184985037251497
-19.184985037251575
-19.184985037251383
-19.1849850372515
-19.184985037251526
-19.184985037251476
-19.18498503725175
-19.184985037251437
-19.184985037251494
-19.18498503725147
-19.18498503725151
-19.184985037251614
-19.184985037251614
-19.18498503725159
-19.184985037251444
-19.184985037251604
-19.184985037251682
-19.18498503725146
-19.18498503725166
-19.184985037251398
-19.18498503725146
-19.18498503725167
-19.184985037251447
-19.184985037251565
-19.18498503725168
-19.18498503725155
-19.184985037251575
-19.184985037251614
-19.184985037251636
-19.184985037251852
-19.18498503725156
-

KeyboardInterrupt: 

In [22]:
EnergyFromCirc(paramsFromLast)-Eg

-19.184985037251607


3.6175791251480405e-09